# Phase 4: Modeling Pipeline

Train and evaluate machine learning models for smoking cessation prediction.

## 1. Setup and Load Data

In [ ]:
# Import libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import our modules
from src.modeling import (
    split_data_by_person,
    train_logistic_regression,
    train_random_forest,
    train_xgboost
)
from src.evaluation import (
    evaluate_model,
    print_evaluation_report,
    plot_roc_curve,
    plot_precision_recall_curve,
    plot_confusion_matrix
)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ Libraries imported")

## 2. Load Engineered Features

In [ ]:
# Load the engineered sample dataset from Phase 3
data_path = Path('../data/processed/engineered_phase3_sample.parquet')
df = pd.read_parquet(data_path)

print(f"Loaded dataset: {df.shape[0]} rows × {df.shape[1]} features")
print(f"\nFeatures: {list(df.columns)}")

## 3. Create Synthetic Target Variable

Since we're working with a sample and don't have actual quit outcomes yet, let's create a synthetic target for demonstration.

In [ ]:
# Create synthetic quit_success outcome based on features
# Higher success probability for: lower dependence, used methods, higher motivation
np.random.seed(42)

# Base probability
base_prob = 0.3

# Adjust based on features
prob = np.full(len(df), base_prob)

# Lower dependence increases success
if 'high_dependence' in df.columns:
    prob = np.where(df['high_dependence'] == 0, prob + 0.15, prob - 0.1)

# Using any method increases success
if 'used_any_method' in df.columns:
    prob = np.where(df['used_any_method'] == 1, prob + 0.2, prob)

# Clip probabilities
prob = np.clip(prob, 0.05, 0.95)

# Generate binary outcomes
df['quit_success'] = np.random.binomial(1, prob)

# Add person_id for proper splitting
df['person_id'] = range(len(df))

print(f"Created synthetic target variable")
print(f"Quit success rate: {df['quit_success'].mean():.3f}")
print(f"\nClass balance:")
print(df['quit_success'].value_counts())

## 4. Prepare Feature List

In [ ]:
# Get all feature columns (exclude target and ID)
exclude_cols = ['quit_success', 'person_id', 'race_ethnicity']  # race_ethnicity is string, use dummies instead
feature_cols = [col for col in df.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features for modeling:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

## 5. Train/Validation/Test Split

Split by person to prevent data leakage.

In [ ]:
# Split data: 60% train, 20% val, 20% test
X_train, X_val, X_test, y_train, y_val, y_test, train_ids, val_ids, test_ids = split_data_by_person(
    df,
    feature_cols=feature_cols,
    test_size=0.4,
    val_size=0.5,
    random_state=42
)

## 6. Train Models

### 6.1 Logistic Regression (Baseline)

In [ ]:
print("Training Logistic Regression...")
lr_model, lr_scaler, y_val_pred_lr, y_val_proba_lr = train_logistic_regression(
    X_train, y_train, X_val, y_val
)
print("✓ Logistic Regression trained")

### 6.2 Random Forest

In [ ]:
print("Training Random Forest...")
rf_model, y_val_pred_rf, y_val_proba_rf = train_random_forest(
    X_train, y_train, X_val, y_val
)
print("✓ Random Forest trained")

### 6.3 XGBoost

In [ ]:
print("Training XGBoost...")
xgb_model, y_val_pred_xgb, y_val_proba_xgb = train_xgboost(
    X_train, y_train, X_val, y_val
)
print("✓ XGBoost trained")

## 7. Evaluate Models

In [ ]:
# Evaluate all models
lr_metrics = evaluate_model(y_val, y_val_pred_lr, y_val_proba_lr, "Logistic Regression")
rf_metrics = evaluate_model(y_val, y_val_pred_rf, y_val_proba_rf, "Random Forest")
xgb_metrics = evaluate_model(y_val, y_val_pred_xgb, y_val_proba_xgb, "XGBoost")

# Print reports
print_evaluation_report(lr_metrics)
print("\n")
print_evaluation_report(rf_metrics)
print("\n")
print_evaluation_report(xgb_metrics)

## 8. Model Comparison Table

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame([
    lr_metrics,
    rf_metrics,
    xgb_metrics
])

# Select key metrics
display_cols = ['model', 'roc_auc', 'pr_auc', 'precision', 'recall', 'f1']
comparison_table = comparison_df[display_cols].round(3)

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison_table.to_string(index=False))

# Identify best model
best_model_idx = comparison_df['roc_auc'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'model']
print(f"\n🏆 Best model (by ROC-AUC): {best_model_name}")

## 9. Visualization: ROC Curves

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

plot_roc_curve(y_val, y_val_proba_lr, "Logistic Regression", ax=ax)
plot_roc_curve(y_val, y_val_proba_rf, "Random Forest", ax=ax)
plot_roc_curve(y_val, y_val_proba_xgb, "XGBoost", ax=ax)

plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Visualization: Precision-Recall Curves

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

plot_precision_recall_curve(y_val, y_val_proba_lr, "Logistic Regression", ax=ax)
plot_precision_recall_curve(y_val, y_val_proba_rf, "Random Forest", ax=ax)
plot_precision_recall_curve(y_val, y_val_proba_xgb, "XGBoost", ax=ax)

plt.title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 11. Feature Importance (XGBoost)

In [ ]:
# Get feature importance from XGBoost
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
top_n = min(20, len(importance_df))
fig, ax = plt.subplots(figsize=(10, 8))

sns.barplot(
    data=importance_df.head(top_n),
    x='importance',
    y='feature',
    palette='viridis',
    ax=ax
)

ax.set_title(f'Top {top_n} Feature Importances (XGBoost)', fontsize=14, fontweight='bold')
ax.set_xlabel('Importance Score')
ax.set_ylabel('Feature')

plt.tight_layout()
plt.show()

print(f"\nTop 10 Most Important Features:")
for idx, row in importance_df.head(10).iterrows():
    print(f"  {row['feature']:30s} {row['importance']:.4f}")

## 12. Confusion Matrices

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

plot_confusion_matrix(y_val, y_val_pred_lr, "Logistic Regression", ax=axes[0])
plot_confusion_matrix(y_val, y_val_pred_rf, "Random Forest", ax=axes[1])
plot_confusion_matrix(y_val, y_val_pred_xgb, "XGBoost", ax=axes[2])

plt.tight_layout()
plt.show()

## 13. Save Best Model

In [ ]:
from src.modeling import save_model

# Save the best performing model
model_dir = Path('../models')
model_dir.mkdir(exist_ok=True)

# Determine best model
best_metrics = comparison_df.loc[best_model_idx]
if best_model_name == 'Logistic Regression':
    best_model = lr_model
    model_path = model_dir / 'logistic_regression_best.pkl'
    # Also save scaler
    save_model(lr_scaler, model_dir / 'logistic_regression_scaler.pkl')
elif best_model_name == 'Random Forest':
    best_model = rf_model
    model_path = model_dir / 'random_forest_best.pkl'
else:
    best_model = xgb_model
    model_path = model_dir / 'xgboost_best.pkl'

# Save model with metadata
metadata = {
    'model_name': best_model_name,
    'feature_cols': feature_cols,
    'metrics': best_metrics.to_dict(),
    'train_size': len(X_train),
    'val_size': len(X_val)
}

save_model(best_model, model_path, metadata)
print(f"\n✓ Best model saved: {best_model_name}")

## Summary

### What We Accomplished

1. ✅ Loaded engineered features from Phase 3
2. ✅ Created proper train/validation/test splits (60/20/20)
3. ✅ Trained three models:
   - Logistic Regression (baseline with feature scaling)
   - Random Forest (ensemble method)
   - XGBoost (gradient boosting)
4. ✅ Evaluated with comprehensive metrics (ROC-AUC, PR-AUC, Precision, Recall, F1)
5. ✅ Visualized performance with ROC and PR curves
6. ✅ Analyzed feature importance
7. ✅ Saved best performing model

### Next Steps

1. **Process Full Dataset**: Move beyond 100-row sample to full PATH data
2. **Hyperparameter Tuning**: Use GridSearch or RandomSearch for optimization
3. **Cross-Validation**: Implement k-fold CV for more robust evaluation
4. **Additional Features**: Search for quit history, motivation, environmental variables
5. **Final Evaluation**: Test best model on held-out test set
6. **Interpretability**: Add SHAP values or other explainability methods